In [4]:
import pandas as pd
import zipfile

In [5]:
# you can download the zip file through the zip provided in this repository
zip_file = 'london_bike_sharing_db.zip' 

In [6]:
# extract the file from the downloaded zip file
with zipfile.ZipFile(zip_file, 'r') as file:
    file.extractall()

In [7]:
# read in the csv file as a pandas dataframe
df = pd.read_csv('london_merged.csv')

### exploring the data 

In [8]:
df.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [9]:
df.dtypes

timestamp        object
cnt               int64
t1              float64
t2              float64
hum             float64
wind_speed      float64
weather_code    float64
is_holiday      float64
is_weekend      float64
season          float64
dtype: object

In [10]:
df.shape

(17414, 10)

In [11]:
df.columns

Index(['timestamp', 'cnt', 't1', 't2', 'hum', 'wind_speed', 'weather_code',
       'is_holiday', 'is_weekend', 'season'],
      dtype='object')

In [12]:
# are there any null values ?
df.isna().sum()

timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
dtype: int64

seems like there is none 

# the column names are not much representative , we are going to modify some

In [14]:
column_names = {
    'cnt': 'new_bike_shares', 
    't1':'temprature_in_C',
    't2':'Felt_temprature', 
    'hum':'humidity',
    'weather_code': 'weather_state'
}
df.rename(columns = column_names, inplace=True)

In [16]:
df.humidity = df.humidity/ 100

# Formatting

## seasons

In [17]:
# that is one way of doing it
#convert seasons code into meaninful season names
seasons = ['spring', 'summer','fall', 'winter']
def season(column):
    if column == 0.0:
        return seasons[0]
    elif column == 1.0:
        return seasons[1]
    elif column == 2.0:
        return seasons[2]
    else:
        return seasons[3]
    
    
df['season'] = df['season'].apply(season)

## weather state

In [18]:
# Convert weather codes into strings
weather = {
    1.0: 'clear',
    2.0: 'few_clouds',
    3.0: 'broken_clouds',
    4.0: 'cloudy',
    7.0: 'Rainy/light rain',
    10.0: ' rain with thunderstorm ',
    26.0: 'snowfall',
    94.0: 'Freezing Frog'
}

# Replace values in the "weather_code" column
df["weather_state"].replace(weather, inplace=True)


## day type

In [19]:
#summarize the two columns 'is_holiday' and 'is_weekend ' into one column which is 'day_type'
def day_type(row):
    if row['is_holiday'] == 0.0 and row['is_weekend'] == 0.0:
        return 'weekday'
    elif row['is_holiday'] == 1.0:
        return 'holiday'
    else:
        return 'weekend'

df['day_type'] = df.apply(day_type, axis=1)


In [20]:
df['day_type'].value_counts()

day_type
weekday    12060
weekend     4970
holiday      384
Name: count, dtype: int64

In [21]:
df.dtypes

timestamp           object
new_bike_shares      int64
temprature_in_C    float64
Felt_temprature    float64
humidity           float64
wind_speed         float64
weather_state       object
is_holiday         float64
is_weekend         float64
season              object
day_type            object
dtype: object

In [22]:
df['season'] = df['season'].astype('str')

In [23]:
df['weather_state'] = df['weather_state'].astype('str')

In [24]:
df['day_type'] = df['day_type'].astype('str')

In [25]:
df

,timestamp,new_bike_shares,temprature_in_C,Felt_temprature,humidity,wind_speed,weather_state,is_holiday,is_weekend,season,day_type
0,2015-01-04 00:00:00,182,3.0,2.0,0.00930,6.0,broken_clouds,0.0,1.0,winter,weekend
1,2015-01-04 01:00:00,138,3.0,2.5,0.00930,5.0,clear,0.0,1.0,winter,weekend
2,2015-01-04 02:00:00,134,2.5,2.5,0.00965,0.0,clear,0.0,1.0,winter,weekend
3,2015-01-04 03:00:00,72,2.0,2.0,0.01000,0.0,clear,0.0,1.0,winter,weekend
4,2015-01-04 04:00:00,47,2.0,0.0,0.00930,6.5,clear,0.0,1.0,winter,weekend
...,...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,0.00810,19.0,broken_clouds,0.0,0.0,winter,weekday
17410,2017-01-03 20:00:00,541,5.0,1.0,0.00810,21.0,cloudy,0.0,0.0,winter,weekday
17411,2017-01-03 21:00:00,337,5.5,1.5,0.00785,24.0,cloudy,0.0,0.0,winter,weekday
17412,2017-01-03 22:00:00,224,5.5,1.5,0.00760,23.0,cloudy,0.0,0.0,winter,weekday


In [26]:
df

,timestamp,new_bike_shares,temprature_in_C,Felt_temprature,humidity,wind_speed,weather_state,is_holiday,is_weekend,season,day_type
0,2015-01-04 00:00:00,182,3.0,2.0,0.00930,6.0,broken_clouds,0.0,1.0,winter,weekend
1,2015-01-04 01:00:00,138,3.0,2.5,0.00930,5.0,clear,0.0,1.0,winter,weekend
2,2015-01-04 02:00:00,134,2.5,2.5,0.00965,0.0,clear,0.0,1.0,winter,weekend
3,2015-01-04 03:00:00,72,2.0,2.0,0.01000,0.0,clear,0.0,1.0,winter,weekend
4,2015-01-04 04:00:00,47,2.0,0.0,0.00930,6.5,clear,0.0,1.0,winter,weekend
...,...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,0.00810,19.0,broken_clouds,0.0,0.0,winter,weekday
17410,2017-01-03 20:00:00,541,5.0,1.0,0.00810,21.0,cloudy,0.0,0.0,winter,weekday
17411,2017-01-03 21:00:00,337,5.5,1.5,0.00785,24.0,cloudy,0.0,0.0,winter,weekday
17412,2017-01-03 22:00:00,224,5.5,1.5,0.00760,23.0,cloudy,0.0,0.0,winter,weekday


## importing the prepared data into an excel file

In [27]:
df.to_excel('london_bikes.xlsx', sheet_name = 'bikes_data')